In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Number of samples
n_samples = 10000

# Initialize data dictionary
data = {}

# Demographic variables
data['age'] = np.clip(np.random.normal(50, 15, n_samples), 18, 100).astype(int)
data['sex'] = np.random.choice([0, 1], n_samples, p=[0.5, 0.5])  # 0=Female, 1=Male
data['family_history'] = np.random.choice([0, 1], n_samples, p=[0.8, 0.2])

# Target: Cancer diagnosis
data['cancer'] = np.random.choice([0, 1], n_samples, p=[0.7, 0.3])

# Symptoms (conditional on cancer)
data['fatigue'] = np.where(
    data['cancer'] == 1,
    np.random.choice([0, 1], n_samples, p=[0.4, 0.6]),
    np.random.choice([0, 1], n_samples, p=[0.7, 0.3])
)
data['weight_loss'] = np.where(
    data['cancer'] == 1,
    np.random.choice([0, 1], n_samples, p=[0.6, 0.4]),
    np.random.choice([0, 1], n_samples, p=[0.9, 0.1])
)
data['pain'] = np.where(
    data['cancer'] == 1,
    np.random.choice([0, 1], n_samples, p=[0.5, 0.5]),
    np.random.choice([0, 1], n_samples, p=[0.8, 0.2])
)
data['fever'] = np.where(
    data['cancer'] == 1,
    np.random.choice([0, 1], n_samples, p=[0.7, 0.3]),
    np.random.choice([0, 1], n_samples, p=[0.9, 0.1])
)
data['night_sweats'] = np.where(
    data['cancer'] == 1,
    np.random.choice([0, 1], n_samples, p=[0.75, 0.25]),
    np.random.choice([0, 1], n_samples, p=[0.95, 0.05])
)
data['bleeding'] = np.where(
    data['cancer'] == 1,
    np.random.choice([0, 1], n_samples, p=[0.7, 0.3]),
    np.random.choice([0, 1], n_samples, p=[0.95, 0.05])
)
data['lumps'] = np.where(
    data['cancer'] == 1,
    np.random.choice([0, 1], n_samples, p=[0.65, 0.35]),
    np.random.choice([0, 1], n_samples, p=[0.9, 0.1])
)
data['cough'] = np.where(
    data['cancer'] == 1,
    np.random.choice([0, 1], n_samples, p=[0.6, 0.4]),
    np.random.choice([0, 1], n_samples, p=[0.85, 0.15])
)
data['bowel_bladder_changes'] = np.where(
    data['cancer'] == 1,
    np.random.choice([0, 1], n_samples, p=[0.7, 0.3]),
    np.random.choice([0, 1], n_samples, p=[0.9, 0.1])
)

# Severity indicators
data['pain_severity'] = np.where(
    data['cancer'] == 1,
    np.clip(np.random.normal(6, 2, n_samples), 0, 10).astype(int),
    np.clip(np.random.normal(3, 2, n_samples), 0, 10).astype(int)
)
data['weight_loss_amount'] = np.where(
    data['cancer'] == 1,
    np.random.exponential(5, n_samples),
    np.random.uniform(0, 2, n_samples)
)
data['bleeding_severity'] = np.where(
    data['cancer'] == 1,
    np.random.choice([0, 1, 2, 3], n_samples, p=[0.3, 0.3, 0.2, 0.2]),
    np.random.choice([0, 1, 2, 3], n_samples, p=[0.95, 0.04, 0.01, 0.0])
)
data['vital_sign_abnormalities'] = np.where(
    data['cancer'] == 1,
    np.random.choice([0, 1], n_samples, p=[0.7, 0.3]),
    np.random.choice([0, 1], n_samples, p=[0.95, 0.05])
)

# Target: Emergency status (based on cancer and severity)
data['emergency'] = np.where(
    (data['cancer'] == 1) & (
        (data['pain_severity'] >= 8) | 
        (data['bleeding_severity'] >= 2) | 
        (data['vital_sign_abnormalities'] == 1)
    ),
    np.random.choice([0, 1], n_samples, p=[0.2, 0.8]),  # 80% emergency if criteria met
    np.random.choice([0, 1], n_samples, p=[0.99, 0.01])  # 1% emergency otherwise
)

In [4]:
# Create DataFrame for coded data
coded_df = pd.DataFrame(data)

# Save coded data
coded_df.to_csv('coded_data.csv', index=False)
print("Coded dataset saved as 'coded_data.csv'")

Coded dataset saved as 'coded_data.csv'


In [8]:
# Display first few rows for verification
print("\nFirst 5 rows of coded_data.csv:")
print(coded_df.head())


First 5 rows of coded_data.csv:
   age  sex  family_history  cancer  fatigue  weight_loss  pain  fever  \
0   57    0               0       0        1            0     0      0   
1   47    0               0       0        1            1     0      1   
2   59    0               1       0        0            0     0      1   
3   72    0               0       0        1            0     0      0   
4   46    1               0       0        1            0     0      0   

   night_sweats  bleeding  lumps  cough  bowel_bladder_changes  pain_severity  \
0             0         0      0      0                      0              3   
1             0         0      0      0                      0              1   
2             0         0      0      0                      0              5   
3             0         0      0      0                      0              2   
4             0         0      0      0                      0              6   

   weight_loss_amount  bleeding_sev

In [5]:
# Create raw data by converting binary/categorical variables to strings
raw_df = coded_df.copy()

# Convert binary variables to Yes/No
binary_columns = [
    'family_history', 'cancer', 'fatigue', 'weight_loss', 'pain', 'fever', 
    'night_sweats', 'bleeding', 'lumps', 'cough', 'bowel_bladder_changes', 
    'vital_sign_abnormalities', 'emergency'
]
for col in binary_columns:
    raw_df[col] = raw_df[col].map({0: 'No', 1: 'Yes'})

# Convert sex to Male/Female
raw_df['sex'] = raw_df['sex'].map({0: 'Female', 1: 'Male'})

# Convert bleeding_severity to None/Mild/Moderate/Severe
raw_df['bleeding_severity'] = raw_df['bleeding_severity'].map({
    0: 'None', 1: 'Mild', 2: 'Moderate', 3: 'Severe'
})

# Rename weight_loss_amount to include unit
raw_df = raw_df.rename(columns={'weight_loss_amount': 'weight_loss_amount_kg'})

In [6]:
# Save raw data
raw_df.to_csv('raw_data.csv', index=False)
print("Raw dataset saved as 'raw_data.csv'")

Raw dataset saved as 'raw_data.csv'


In [9]:
# Display first few rows for verification
print("\nFirst 5 rows of raw_data.csv:")
print(raw_df.head())


First 5 rows of raw_data.csv:
   age     sex family_history cancer fatigue weight_loss pain fever  \
0   57  Female             No     No     Yes          No   No    No   
1   47  Female             No     No     Yes         Yes   No   Yes   
2   59  Female            Yes     No      No          No   No   Yes   
3   72  Female             No     No     Yes          No   No    No   
4   46    Male             No     No     Yes          No   No    No   

  night_sweats bleeding lumps cough bowel_bladder_changes  pain_severity  \
0           No       No    No    No                    No              3   
1           No       No    No    No                    No              1   
2           No       No    No    No                    No              5   
3           No       No    No    No                    No              2   
4           No       No    No    No                    No              6   

   weight_loss_amount_kg bleeding_severity vital_sign_abnormalities emergency  
0    